In [1]:
import pandas as pd
import numpy as np
import json
import time
import csv

# Récupération de la playlist dans un fichier csv

In [2]:
# FOnction pour calculer le temps de récupération
def convert_temps(t):
    nb_min, nb_sec = str(round(t//60)), str(round(t%60))
    if len(nb_sec) == 1:
        nb_sec = '0' + nb_sec
    return (nb_min + "m " + nb_sec + "s")

In [3]:
index_csv=['playlist',
           'track_uri']

In [4]:
# Récupération des playlist dans les dictionnaires json et enregistrement dans un fichier csv
start = time.time()
with open('Data/playlist_spotify_spare.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = index_csv)
    writer.writeheader()
    for i in range(1000):
        name='Data/mpd.slice.'+str(i*1000)+'-'+str(i*1000+999)+'.json'
        with open(name) as f: 
            playlists = json.load(f)['playlists'] # Get playlist info
            for j, p in enumerate(playlists): # Iterate through each playlist
                for song in p['tracks']:
                    writer.writerow({
                    'playlist':i * 1000 + j,
                    'track_uri':song['track_uri']
                    })


end = time.time()
diff = end - start
print("Temps d'éxécution :",convert_temps(diff))

Temps d'éxécution : 8m 27s


# Nettoyage des données

In [5]:
# Chargement des donnée extrait dans un csv
playlist=pd.read_csv('Data/playlist_spotify_spare.csv')
playlist.head()

,playlist,track_uri
0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,0,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,0,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,0,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,0,spotify:track:1lzr43nnXAijIGYnCT8M8H


In [6]:
# Fonction utilisé ensuite pour supprimer "spotify:track:" des lignes de la colonne track_uri de playlist
def reduc_track(col):
    return col[14:]


In [7]:
playlist['track_uri']=playlist['track_uri'].apply(reduc_track)

In [8]:
playlist.head()

,playlist,track_uri
0,0,0UaMYEvWZi0ZqiDOoHU3YI
1,0,6I9VzXrHxO9rA9A5euc8Ak
2,0,0WqIKmW4BTrj3eJFmnCKMv
3,0,1AWQoqb9bSvzTjaLralEkT
4,0,1lzr43nnXAijIGYnCT8M8H


In [9]:
# Sauvegarde
playlist.to_csv('Data/playlist_spotify_spare.csv')

# Fusion avec les caractéristiques des musiques

In [10]:
musique=pd.read_csv('Data/musique_spotify.csv',index_col=0)
musique.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,popularity,genre
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02,65,"alternative metal, alternative rock, conscious..."
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02,69,"alternative metal, alternative rock, conscious..."
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02,55,"alternative metal, alternative rock, conscious..."
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02,50,"alternative metal, alternative rock, conscious..."
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02,66,"alternative metal, alternative rock, conscious..."


In [11]:
playlist=playlist.rename(columns={'track_uri':'id'})

In [12]:
musique_playlist=musique.merge(playlist[['id']],on='id').drop_duplicates()
musique_playlist.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,popularity,genre
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02,65,"alternative metal, alternative rock, conscious..."
1544,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02,69,"alternative metal, alternative rock, conscious..."
3351,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02,55,"alternative metal, alternative rock, conscious..."
3660,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02,50,"alternative metal, alternative rock, conscious..."
3730,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02,66,"alternative metal, alternative rock, conscious..."


In [13]:
# Jointure interne des deux dataframe
playlist_musique_spotify=playlist.merge(musique_playlist[['id']],on='id')
playlist_musique_spotify.head()

,playlist,id
0,0,6I9VzXrHxO9rA9A5euc8Ak
1,38,6I9VzXrHxO9rA9A5euc8Ak
2,123,6I9VzXrHxO9rA9A5euc8Ak
3,262,6I9VzXrHxO9rA9A5euc8Ak
4,355,6I9VzXrHxO9rA9A5euc8Ak


# Netoyage des données

In [14]:
# Determination du nombre de musique par playlist dans la jointure
playlist_1track=playlist_musique_spotify.groupby(['playlist']).agg({'id':pd.Series.count})
playlist_1track=playlist_1track.rename(columns={'id':'Nbr_music'})
playlist_1track

,Nbr_music
playlist,
0,6
1,12
2,3
3,11
4,4
...,...
999993,7
999994,6
999995,6


In [15]:
playlist_musique_spotify=playlist_musique_spotify.merge(playlist_1track.reset_index(),on='playlist')
playlist_musique_spotify.head()

,playlist,id,Nbr_music
0,0,6I9VzXrHxO9rA9A5euc8Ak,6
1,0,1AWQoqb9bSvzTjaLralEkT,6
2,0,7H6ev70Weq6DdpZyyTmUXk,6
3,0,2PpruBYCo4H7WOBJ7Q2EwM,6
4,0,4pmc2AxSEq6g7hPVlJCPyP,6


In [16]:
playlist_musique_spotify.shape

(8282872, 3)

In [17]:
# Suppresion des playlist avec une seule musique (inutile pour la recommendation)
playlist_musique_spotify=playlist_musique_spotify[['playlist','id']][playlist_musique_spotify['Nbr_music']>1]
playlist_musique_spotify.head()

,playlist,id
0,0,6I9VzXrHxO9rA9A5euc8Ak
1,0,1AWQoqb9bSvzTjaLralEkT
2,0,7H6ev70Weq6DdpZyyTmUXk
3,0,2PpruBYCo4H7WOBJ7Q2EwM
4,0,4pmc2AxSEq6g7hPVlJCPyP


In [18]:
playlist_musique_spotify.shape

(8182576, 2)

In [19]:
musique_playlist.shape

(134712, 26)

In [20]:
# Réduction du DataFrame musique_spotify au musique ayant des playlist
musique_playlist=playlist_musique_spotify[['id']].merge(musique_playlist,on='id').drop_duplicates()

In [21]:
musique_playlist.shape

(133642, 26)

In [22]:
musique_playlist.to_csv('Data/musique_playlist.csv')
playlist_musique_spotify.to_csv('Data/playlist_musique_spotify.csv')